In [1]:
from elasticsearch import Elasticsearch
import json
import pandas as pd
import rocksdb


es = Elasticsearch(hosts="https://localhost:9200", basic_auth=["elastic", "2qsz7K*6lxelAyO*lV=*"], verify_certs=False)

index = ".ds-winlogbeat-8.8.2-2023.08.06-000001"
body = {
    "query": {
        "match": {
            "event.code": "1"
        }
    },
    "size": 1000,
    "_source": ["message"]
}

res = es.search(index=index, body=body)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/elasticsearch/_sync/client/__init__.py:395: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(
/var/folders/ly/h9mjdl311vl8hjq1r10v8r200000gn/T/ipykernel_81041/860402210.py:20: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(index=index, body=body)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [2]:
# define the substrings to be replaced
replace_strings = ["Process Create:", "RuleName: ", "UtcTime: ","FileVersion: ", "Description: ",
"Product: ","Company: ","OriginalFileName: ","CurrentDirectory: ", "LogonGuid: ", "LogonId: ", "TerminalSessionId: ",
"IntegrityLevel: ", "Hashes: ", "ParentProcessGuid: ", "ParentProcessId: ", "ParentImage: ", "ParentCommandLine: ", "ParentUser: ",
"ProcessGuid: ", "ProcessId: ", "Image: ", "CommandLine: "]

# store messages in a list
messages = []
for hit in res['hits']['hits']:
    message = hit["_source"].get("message")
    if message is not None:
        # remove the unwanted substrings
        for s in replace_strings:
            message = message.replace(s, "")
        messages.append(message)

# create a DataFrame
df = pd.DataFrame([message.split('\n') for message in messages])


In [3]:
df.columns = ['EventType', 'RuleName', 'UtcTime', 'ProcessGuid', 'ProcessId', 'Image', 'FileVersion', 'Description', 'Product', 'Company', 'OriginalFileName', 'CommandLine',
'CurrentDirectory', 'User', 'LogonGuid', 'LogonId', 'TerminalSessionId', 'IntegrityLevel', 'Hashes', 'ParentProcessGuid', 'ParentProcessId', 'ParentImage', 'ParentCommandLine', 'ParentUser']

In [4]:
df.EventType = "Process Create"

In [5]:
db = rocksdb.RocksDB("/Users/dong-ju/Dropbox/EINSIS/03. CODE/files/rocksdb/python", rocksdb.Options(create_if_missing=True))
